A script to analyze a diary file day-by-day with a local large language model (LLM)
See https://www.transformingmed.tech/p/decoding-happiness-an-ai-driven-experiment
Matthijs Cluitmans for Transforming Med.Tech (http://transformingmed.tech/)
Steps:
- Install LM Studio (https://lmstudio.ai/)
- In LM Studio, find a suitable LLM that fits with the specs of your computer. LM Studio will tell you if it will run or not. I used the Vicuna LLM (vicuna-13b-v1.5-16k.Q5_K_M.gguf) from Hugging Face https://huggingface.co/TheBloke/vicuna-13B-v1.5-16K-GGUF
- In LM Studio, go to Local Inference Server, load the model (with the appropriate preset, e.g. Vicuna 1.5 16k), and start the server
- Than run this Jupyter notebook with your local copy of diary.csv, which contains two columns: Date and Tekst, semicolon separated (;) and is an UTF-8 file


In [63]:
# Matthijs Cluitmans for Transforming Med.Tech (http://transformingmed.tech/)


import openai
import csv

# Put your URI end point:port here for your local inference server (in LM Studio) 
openai.api_base='http://localhost:1234/v1'
# Put in an empty API Key
openai.api_key=''

# Adjust the following based on the model type
# Alpaca style prompt format (suitable for Vicuna):
prefix = "### Instruction:\n" 
suffix = "\n### Response:"

# 'Llama2 Chat' prompt format (required for some other LLMs):
#prefix = "[INST]"
#suffix = "[/INST]"

temperature = 0 # Vary the temperature if needed; but higher temp gives more hallucinations. I had best results at 0

# Simple wrapper function for prompts
def get_completion(prompt, messages, model="local model", temperature=0.0):
    formatted_prompt = f"{prefix}{prompt}{suffix}"
    messages.extend([{"role": "user", "content": formatted_prompt}])
    #print(messages)
    #print(f'\nYour prompt: {prompt}\n')
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        n_predict=-1,
        repeat_penalty=1.1
    )
    return response.choices[0].message["content"]

# These are the prompts I run on every dairy input. Adapt to your needs (e.g., now it assumes Dutch input but will respond in English.)
systemprompt = 'You are a thorough and truthful assistant that analyses psychological content of a diary.'
prompts = []
prompts.append('I will give you a part of a diary in Dutch. We\'re going to analyse the elements that display happiness or unhappiness. We\'ll do that in a few steps. First, please extract a single "happiness score" from 1-10 that reflects how happy the user seems to be that day, 1 being very unhappy and 10 being very happy. Only provide number as output, nothing else, so just a single number. If you do not have sufficient information, respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please extract a single "gratefulness score" from 1-10 that reflects how grateful the user seems to be that day, 1 being not grateful at all and 10 being very grateful. Only provide number as output, nothing else, so just a single number. If you do not have sufficient information, respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, provide a few summarizing keywords (in English, separated by commas) that characterize the main experiences that made the user happy. Keep your response short. If you do not have sufficient information, respond "blank" and do not make up content that was not provided by the user. Only provide the keywords as output, nothing else. Respond in English.')
prompts.append('Now, based on the diary, provide a few summarizing keywords (in English, separated by commas) that characterize the main experiences that made the user unhappy. Keep your response short. If you do not have sufficient information, respond "blank" and do not make up content that was not provided by the user. Only provide the keywords as output, nothing else. Respond in English.')
prompts.append('Now, based on the diary, please extract a score from 1-5 for the weather that day, 1 being very bad weather, 5 being very good weather. Only provide number as output, nothing else, so just a single number. If you do not have information about the weather (which may be likely), respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please extract a score from 1-5 for how busy the user was that day, 1 being not busy at all, 5 being extremely busy. Only provide number as output, nothing else, so just a single number. If you do not have information about busyness (which may be likely), respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please extract a score from 1-5 for how the user enjoyed spending time with family that day, 1 indicating not at all, 5 indicating a lot. Only provide number as output, nothing else, so just a single number. If you do not have information about family time (which may be likely), respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please extract a score from 1-5 for how the user enjoyed work that day, 1 indicating not at all, 5 indicating a lot. Only provide number as output, nothing else, so just a single number. If you do not have information about work satisfaction (which may be likely), respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please extract a score from 1-5 for how much family time the user had that day, 1 indicating no family time at all, 5 indicating spending the whole day with the family. Only provide number as output, nothing else, so just a single number. If you do not have information about family (which may be likely), respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please extract a score from 1-5 for how much the health of the user or of his family affected them that day, 1 indicating not at all, 5 indicating a lot. Only provide number as output, nothing else, so just a single number. If you do not have information about family (which may be likely), respond with "blank" only and nothing else (no explanation is required).')
prompts.append('Now, based on the diary, please use a few keywords (in English, separated by commas) that could help determine personality traits that are apparent from this text. If none are available (which may be likely), respond with "blank" only and nothing else (no explanation is required).')


In [66]:
# Read diary, ask prompt, write output to new file
# Create output file: I had to enforce utf-8 and fix some weird emoticons in my diary.
with open('diaryLLMoutput.txt', 'a', encoding="utf-8") as outputfile:
    outputfile.write('\n\n-----------------------------\n NEW RUN\n-----------------------------')
    # Opening the diary file, also as UTF-8 and with ; as delimiter
    with open('diary.csv', newline='', encoding="UTF-8") as csvfile:
        diaryreader = csv.reader(csvfile, delimiter=';')
        line_count = 0
        system_prompt = {"role": "system", "content": systemprompt}
        messages = [system_prompt] # initialize with system prompt
        for row in diaryreader:
            if line_count > 0:
                messages = [system_prompt] # start over again PER DAY (I also tried to retain a few days but this did not improve results considerably)
                response = []
                print('\nProcessing ' + row[0])
                # Remark: processing takes a long time, much longer than what you're expected from cloud-based solutions such as ChatGPT. So be patient.
                for i in range(len(prompts)):
                    if i==0:
                        prompt = prompts[0] + '\n\n' + row[1]
                    else:
                        prompt = prompts[i]
                    # Store the response
                    response.append(get_completion(prompt, messages=messages, temperature=temperature))
                    messages.extend([{"role": "assistant", "content": response[i]}]) 
                # Write the response to the file
                outputfile.write('\n' + row[0] + ';')
                for x in response:
                    outputfile.write(x+ ';')
                outputfile.flush()
            line_count += 1
            


Processing  2022-11-01 

Processing  2022-11-02 


KeyboardInterrupt: 